In [1]:
import pandas as pd
import re

In [2]:
def predicting_txt_to_dataframe(path):    
    with open(path, 'r') as f:
        txt = str(f.read())
        txt_list = txt.split('\n')
    row_list = list()
    tmp_list = list()
    for row in txt_list:
        if row == '--------------------':
            tmp_list[0] = tmp_list[0].replace('article_id:', '')
            tmp_list[0] = int(tmp_list[0])
            row_list.append(tmp_list)
            tmp_list = list()
            continue
        if len(row) > 0:
            tmp_list.append(row)
    df = pd.DataFrame(row_list, columns=['article_id','text'])
    return df

In [3]:
df = predicting_txt_to_dataframe('/home/Danny/AI-CUP-2020/datasets/stage5/test.txt')
df

,article_id,text
0,0,醫師：最近人有沒有什麼不舒服？民眾：沒有。醫師：沒有喔。我們本來說要月，年底才打對不對？民眾...
1,1,醫師：要去上課嗎？民眾：沒有。這學期已經開始不上課了。醫師：啊今天的，呃，應該是昨天的抽血。...
2,2,醫師：後來那個吃藥沒有問題對不對？民眾：沒有。醫師：诶？你是打針的？民眾：對。醫師：阿身上有...
3,3,民眾：我自己昨天開始，一天吃一次一粒。醫師：一次一粒，減半量還好？民眾：阿可是……減到今天早...
4,4,醫師：喔，啊嘉明最近還好嗎？民眾：ＯＫ。醫師：啊今年的流感打了嗎？民眾：還沒。醫師：那我們今...
...,...,...
154,154,醫師：這個月還好嗎？民眾：就……醫師：這個月有去碰嗎？民眾：沒有，不去碰的時候就會覺得慾望非...
155,155,醫師：啊這個月還好嗎？民眾：還好啊。醫師：好，所以你反正天天吃啦齁？民眾：對。醫師：好，我們...
156,156,醫師：好那所以總之就是好，所以總之反正，總之就是……這這……就是反正你剩下的藥你就是……就是...
157,157,醫師：好，所以第一個月吃藥的狀況怎麼樣？民眾：嗯，吃藥狀況喔？醫師：嗯。民眾：就是吃了，一開...


In [4]:
entity_type_list = pd.read_csv('entity_type.csv')['entity_type'].tolist()
entity_type_list

['med_exam',
 'location',
 'time',
 'profession',
 'name',
 'money',
 'organization',
 'family',
 'ID',
 'clinical_event',
 'education',
 'contact',
 'others']

In [5]:
# contact_df = pd.read_csv('ground_truth/contact.csv')
# contact_list = contact_df['entity_text'].tolist()
# contact_list

In [6]:
def get_entity(article_id, text, entity_text, entity_type):
    output_str = str()
    source = '2-dict'
    for match in re.finditer(entity_text, text, re.IGNORECASE):
        output_str += '{}\t{}\t{}\t{}\t{}\t{}\n'.format(article_id, match.start(), match.end(), match.group(), entity_type, source)
    return output_str

In [7]:
output_str = '{}\t{}\t{}\t{}\t{}\t{}\n'.format('article_id', 'start_position', 'end_position', 'entity_text', 'entity_type', 'source')

In [8]:
# ckip
entity_type_list = ['contact', 'education', 'ID', 'location', 'organization', 'profession']
for entity_type in entity_type_list:
    dictionary_path = 'ckip/{}.csv'.format(entity_type)
    entity_text_list = pd.read_csv(dictionary_path)['entity_text'].tolist()
    for entity_text in entity_text_list:
        for article_id, text in zip(df['article_id'], df['text']):
            output_str += get_entity(article_id, text, str(entity_text), entity_type)

In [9]:
# ground_truth
entity_type_list = ['clinical_event', 'contact', 'education', 'family', 'location', 'organization', 'others']
for entity_type in entity_type_list:
    dictionary_path = 'ground_truth/{}.csv'.format(entity_type)
    entity_text_list = pd.read_csv(dictionary_path)['entity_text'].tolist()
    for entity_text in entity_text_list:
        for article_id, text in zip(df['article_id'], df['text']):
            output_str += get_entity(article_id, text, str(entity_text), entity_type)

In [10]:
# online
entity_type_list = ['contact', 'family']
for entity_type in entity_type_list:
    dictionary_path = 'online/{}.csv'.format(entity_type)
    entity_text_list = pd.read_csv(dictionary_path)['entity_text'].tolist()
    for entity_text in entity_text_list:
        for article_id, text in zip(df['article_id'], df['text']):
            output_str += get_entity(article_id, text, str(entity_text), entity_type)

In [11]:
# print(output_str)

In [12]:
with open('dictionary.tsv', 'w', encoding='utf-8') as f:
    f.write(output_str)

In [13]:
df = pd.read_csv('dictionary.tsv', sep='\t')
df = df.drop_duplicates()
df = df.sort_values(by=['article_id', 'start_position', 'end_position'])
df

,article_id,start_position,end_position,entity_text,entity_type,source
81,0,227,229,麻豆,location,2-dict
82,0,237,239,麻豆,location,2-dict
73,5,184,186,榮總,location,2-dict
574,6,2271,2273,阿姨,family,2-dict
499,7,42,43,姊,family,2-dict
...,...,...,...,...,...,...
420,157,1751,1753,泰國,location,2-dict
421,157,1794,1796,泰國,location,2-dict
422,157,2177,2179,泰國,location,2-dict
233,157,2923,2927,LINE,contact,2-dict


In [14]:
df[df.duplicated(keep=False, subset=['article_id', 'start_position'])]

,article_id,start_position,end_position,entity_text,entity_type,source
516,7,1087,1088,媽,family,2-dict
322,7,1087,1089,媽媽,family,2-dict
597,9,860,862,你媽,family,2-dict
593,9,860,863,你媽媽,family,2-dict
518,9,861,862,媽,family,2-dict
...,...,...,...,...,...,...
325,110,437,439,媽媽,family,2-dict
179,110,1381,1383,石門,location,2-dict
176,110,1381,1385,石門水庫,location,2-dict
508,137,53,54,弟,family,2-dict


In [15]:
# 刪除小的 eg: 媽
df = df.drop_duplicates(keep='last', subset=['article_id', 'start_position'])
df

,article_id,start_position,end_position,entity_text,entity_type,source
81,0,227,229,麻豆,location,2-dict
82,0,237,239,麻豆,location,2-dict
73,5,184,186,榮總,location,2-dict
574,6,2271,2273,阿姨,family,2-dict
499,7,42,43,姊,family,2-dict
...,...,...,...,...,...,...
420,157,1751,1753,泰國,location,2-dict
421,157,1794,1796,泰國,location,2-dict
422,157,2177,2179,泰國,location,2-dict
233,157,2923,2927,LINE,contact,2-dict


In [16]:
df[df.duplicated(keep=False, subset=['article_id', 'end_position'])]

,article_id,start_position,end_position,entity_text,entity_type,source
322,7,1087,1089,媽媽,family,2-dict
517,7,1088,1089,媽,family,2-dict
593,9,860,863,你媽媽,family,2-dict
323,9,861,863,媽媽,family,2-dict
519,9,862,863,媽,family,2-dict
...,...,...,...,...,...,...
340,141,2144,2145,爸,family,2-dict
287,142,456,461,EMAIL,contact,2-dict
286,142,457,461,MAIL,contact,2-dict
12,144,120,125,清大電機所,education,2-dict


In [17]:
# 刪除小的 eg: 媽
df = df.drop_duplicates(keep='first', subset=['article_id', 'end_position'])
df

,article_id,start_position,end_position,entity_text,entity_type,source
81,0,227,229,麻豆,location,2-dict
82,0,237,239,麻豆,location,2-dict
73,5,184,186,榮總,location,2-dict
574,6,2271,2273,阿姨,family,2-dict
499,7,42,43,姊,family,2-dict
...,...,...,...,...,...,...
420,157,1751,1753,泰國,location,2-dict
421,157,1794,1796,泰國,location,2-dict
422,157,2177,2179,泰國,location,2-dict
233,157,2923,2927,LINE,contact,2-dict


In [18]:
df.to_csv('dictionary.tsv', sep='\t', index=False)